### 두개의 투자 전략을 하나의 class 선언

1. 클래스의 이름은 Invest 상속은 X
2. 생성자 함수 
    - 매개변수 4개
        - 주식 데이터 (_df)
        - 기준이 되는 컬럼의 이름 (_col = 'Adj Close')
        - 투자의 시작 시간 (_start = '2010-01-01')
        - 투자의 종료 시간 (_end = datetime.now())
    - 결측치, 양의 무한대, 음의 무한대를 제외
    - 컬럼중 Date 컬럼이 존재한다면 Date 컬럼을 인덱스로 변환
    - index를 시계열 데이터로 변환
    - index에 tz에 존재한다면 tz를 None 변경
    - 시작 시간과 종료 시간은 시계열 데이터로 변환
        - 종료시간은 문자열인 경우에만 변환환
    - 기준이 컬럼(self.col), 시작시간(self.start), 종료시간(self.end)은 self변수로 생성
3. buyandhold 함수를 생성 
    - 매개변수 x (self를 생성)
    - 시작시간과 종료시간으로 인덱스 필터 기준이되는 컬럼을 제외하고 모두 제거 
    - 일별 수익율 컬럼을 생성하여 pct_change() + 1 함수를 이용하여 값을 대입
    - 누적수익율 컬럼을 생성하여 누적 수익율 데이터를 대입 
    - 만들어진 데이터프레임과 총 누적수익율을 되돌려준다.
4. bollinger 함수를 생성 
    - 매개변수 2개 
        - 신뢰구간 (_num = 2)
        - 데이터 개수 (_cnt = 20)
    - 특정컬럼(self.col) 컬럼을 제외한 나머지 컬럼을 제거하고 변수에 저장
    - 이동평균선, 상단밴드, 하단밴드 생성
    - 시작시간, 종료 시간을 기준으로 데이터 필터링 
    - 보유 내역 추가 
    - 데이터 프레임을 되돌려준다. 
5. 수익율을 계산하는 함수 생성
    - 복사본 생성 
    - rtn 컬럼을 생성해서 1을 대입 
    - 매수, 매도 시기를 확인하여 수익율 대입 
    - rtn을 기준으로 누적 수익율을 acc_rtn 컬럼에 대입 
    - 만들어진 데이터프레임과 최종수익율을 되돌려준다.


In [1]:
import pandas as pd 
from datetime import datetime
import numpy as np

In [20]:
class Invest:
    # 생성자 함수 
    def __init__(
            self,
            _df, 
            _col = 'Adj Close', 
            _start = '2010-01-01', 
            _end = datetime.now()
    ):
        # 조건식 생성 
        flag = _df.isin( [np.nan, np.inf, -np.inf ] ).any(axis=1)
        self.df = _df.loc[~flag]
        # self.df의 컬럼중 Date가 존재한다면?
        if 'Date' in self.df.columns:
            self.df.set_index('Date', inplace=True)
        # index를 시계열 데이터로 변경 
        # index를 시계열로 변경할때 DatetimeIndex -> tz 가 존재 
        # index를 시계열로 변경할때 Index -> tz 존재 X
        self.df.index = pd.DatetimeIndex(pd.to_datetime(self.df.index))
        # tz가 존재하는가?
        if self.df.index.tz:
            self.df.index = self.df.index.tz_localize(None)
        # 인자값으로 들어온 기준이 되는 컬럼의 이름은 self변수에 저장
        self.col = _col
        # 시작 시간은 시계열로 변경 
        try:
            self.start = datetime.strptime(_start, '%Y-%m-%d')
            if type(_end) == 'str':
                self.end = datetime.strptime(_end, '%Y-%m-%d')
            else:
                self.end = _end
        except:
            print("시간의 타입이 맞지 않습니다. 포멧은 YYYY-mm-dd 형식입니다.")
            print('Class를 다시 생성하시기 바랍니다.')
        
    def buyandhold(self):
        # 시작 시간, 종료시간 , 특정 컬럼을 기준으로 데이터를 필터 
        result = self.df.loc[self.start : self.end , [self.col]]
        # 일별 수익율 생성
        result['rtn'] = (result[self.col].pct_change() + 1).fillna(1)
        # 누적 수익율 생성 
        result['acc_rtn'] = result['rtn'].cumprod()
        # 총 누적 수익율을 변수에 저장 
        acc_rtn  = result.iloc[-1, -1]
        return result, acc_rtn
    
    def bollinger(
            self, 
            _num = 2, 
            _cnt = 20
    ):
        # 특정 컬럼을 기준으로 데이터를 필터
        result = self.df[[self.col]]
        # 이동 평균선 생성
        result['center'] = result[self.col].rolling(_cnt).mean()
        # 표준편차 값 변수에 저장 
        std_series = result[self.col].rolling(_cnt).std()
        # 상단밴드 생성
        result['up'] = result['center'] + (_num * std_series)
        # 하단밴드 생성
        result['down'] = result['center'] - (_num * std_series)
        # 시작 시간과 종료시간을 기준으로 데이터 필터
        result = result.loc[self.start : self.end]
        # 보유 내역 추가 
        result['trade'] = ''
        for idx in result.index:
            # 상단 밴드의 가격보다 기준이 되는 컬럼의 값이 크거나 같다면
            if result.loc[idx, self.col] >= result.loc[idx, 'up']:
                result.loc[idx, 'trade'] = ''
            # 하단밴드의 가격보다 기준이 되는 컬럼의 값이 작거나 같다면
            elif result.loc[idx, 'down'] >= result.loc[idx, self.col]:
                result.loc[idx, 'trade'] = 'buy'
            # 밴드 중간에 기준이되는 컬럼의 값이 존재한다면?
            else:
                # 현재 보유중이면 유지
                if result.shift().loc[idx, 'trade'] == 'buy':
                    result.loc[idx, 'trade'] = 'buy'
                # 보유중이 아니면 유지
                else:
                    result.loc[idx, 'trade'] = ''
        return result



In [15]:
df = pd.read_csv('../../csv/AAPL.csv')

In [21]:
test_invest = Invest(df)

In [17]:
test_invest.buyandhold()

(             Adj Close       rtn   acc_rtn
 Date                                      
 2010-01-04   26.782711  1.000000  1.000000
 2010-01-05   26.829010  1.001729  1.001729
 2010-01-06   26.402260  0.984094  0.985795
 2010-01-07   26.353460  0.998152  0.983973
 2010-01-08   26.528664  1.006648  0.990515
 ...                ...       ...       ...
 2019-06-18  198.449997  1.023518  7.409631
 2019-06-19  197.869995  0.997077  7.387975
 2019-06-20  199.460007  1.008036  7.447342
 2019-06-21  198.779999  0.996591  7.421952
 2019-06-24  199.169998  1.001962  7.436514
 
 [2384 rows x 3 columns],
 7.436513727083075)

In [22]:
test_invest.bollinger()

C:\Users\moons\AppData\Local\Temp\ipykernel_21028\2798864499.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['center'] = result[self.col].rolling(_cnt).mean()
C:\Users\moons\AppData\Local\Temp\ipykernel_21028\2798864499.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['up'] = result['center'] + (_num * std_series)
C:\Users\moons\AppData\Local\Temp\ipykernel_21028\2798864499.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

,Adj Close,center,up,down,trade
Date,,,,,
2010-01-04,26.782711,25.037723,27.046734,23.028713,
2010-01-05,26.829010,25.169503,27.288098,23.050908,
2010-01-06,26.402260,25.307290,27.366449,23.248130,
2010-01-07,26.353460,25.436879,27.410937,23.462821,
2010-01-08,26.528664,25.525609,27.529742,23.521475,
...,...,...,...,...,...
2019-06-18,198.449997,185.432500,201.032574,169.832427,buy
2019-06-19,197.869995,185.996000,202.558154,169.433846,buy
2019-06-20,199.460007,186.830000,204.361771,169.298229,buy
